In [1]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import transformers
# import torch

# model = "tiiuae/falcon-7b"

# tokenizer = AutoTokenizer.from_pretrained(model)

In [2]:
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
# )

In [3]:
# sequences = pipeline(
#    "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
#     max_length=200,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
# )

In [4]:
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")
# #

In [5]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

In [7]:
model_name =  "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("tiiuae/falcon-7b", num_labels=5)

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

2023-11-28 16:23:43.783221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 16:23:46.111499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-28 16:23:46.111662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-28 16:23:46.111679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [13]:
def compute_metrics(eval_pred):
    logits = eval_pred[0]
    labels  = eval_pred[1]
    predictions = np.argmax(logits, axis=-1)
    print(metric.compute(predictions=predictions, references=labels))
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

Trainer is attempting to log a value of "{'AutoConfig': 'tiiuae/falcon-7b--configuration_falcon.FalconConfig', 'AutoModel': 'tiiuae/falcon-7b--modeling_falcon.FalconModel', 'AutoModelForSequenceClassification': 'tiiuae/falcon-7b--modeling_falcon.FalconForSequenceClassification', 'AutoModelForTokenClassification': 'tiiuae/falcon-7b--modeling_falcon.FalconForTokenClassification', 'AutoModelForQuestionAnswering': 'tiiuae/falcon-7b--modeling_falcon.FalconForQuestionAnswering', 'AutoModelForCausalLM': 'tiiuae/falcon-7b--modeling_falcon.FalconForCausalLM'}" for key "auto_map" as a parameter. MLflow's log_param() only accepts values no longer than 250 characters so we dropped this attribute. You can use `MLFLOW_FLATTEN_PARAMS` environment variable to flatten the parameters and avoid this message.


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.44 GiB. GPU 0 has a total capacty of 47.54 GiB of which 1.01 GiB is free. Process 147263 has 22.14 GiB memory in use. Process 146755 has 1.21 GiB memory in use. Including non-PyTorch memory, this process has 23.15 GiB memory in use. Of the allocated memory 22.30 GiB is allocated by PyTorch, and 543.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.predict(small_train_dataset)

{'accuracy': 0.97}


PredictionOutput(predictions=array([[ 0.2460949 , -2.104171  , -2.6782625 , -0.03205827,  5.024416  ],
       [-2.7221694 ,  1.982754  ,  3.7753754 , -0.5633599 , -3.2965202 ],
       [ 0.26904428, -1.9998862 , -2.5836709 , -0.28182426,  4.852385  ],
       ...,
       [ 0.9643748 ,  4.77949   , -0.68747085, -2.4564073 , -2.0013974 ],
       [-2.916281  , -0.28004304,  4.16716   ,  0.92196363, -2.392922  ],
       [-0.6182032 , -2.2214987 , -2.559993  ,  1.6672256 ,  4.862492  ]],
      dtype=float32), label_ids=array([4, 2, 4, 0, 2, 2, 3, 0, 0, 4, 0, 4, 3, 1, 3, 0, 4, 3, 4, 2, 0, 4,
       0, 1, 0, 1, 1, 2, 4, 3, 3, 2, 4, 3, 0, 2, 2, 0, 1, 2, 2, 2, 0, 1,
       4, 1, 1, 2, 0, 2, 1, 1, 2, 4, 1, 3, 2, 4, 3, 3, 3, 1, 0, 3, 3, 2,
       1, 0, 0, 0, 3, 2, 0, 3, 4, 4, 0, 1, 3, 1, 0, 3, 1, 4, 1, 3, 2, 3,
       2, 4, 3, 3, 1, 4, 0, 2, 4, 1, 4, 3, 3, 3, 0, 2, 4, 0, 4, 4, 2, 1,
       4, 3, 4, 1, 2, 0, 3, 0, 0, 0, 4, 3, 1, 0, 3, 4, 0, 4, 1, 4, 0, 3,
       0, 1, 2, 3, 0, 1, 2, 2, 2, 1, 0, 2, 3

In [ ]:
trainer.compute_metrics(small_eval_dataset)

TypeError: object of type 'numpy.int64' has no len()

In [ ]:
# small_train_dataset.column_names

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
# from haystack.nodes import FARMReader

# reader = FARMReader(model_name_or_path="tiiuae/falcon-7b", use_gpu=True)

2023-10-17 23:45:46.045266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 23:45:47.176613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-17 23:45:47.176735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-17 23:45:47.176751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/shaurya/snap/snapd-desktop-integration/83/Desktop/dolly/falconenv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of FalconForQuestionAnswering were not initialized from the model checkpoint at tiiuae/falcon-7b and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 316.00 MiB (GPU 0; 47.54 GiB total capacity; 14.40 GiB already allocated; 244.12 MiB free; 14.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF